<a href="https://colab.research.google.com/github/s-nagesh-livescore/colab-notebooks/blob/main/VB_FBB_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Big Query Connection
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

import pandas as pd
import matplotlib.pyplot as plt

project = 'gamesys-eu-live-sbtech-insight'
location = 'EU'
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

In [ ]:
#@title Defining Parameters
DOMAIN = ["VirginBet", "LiveScoreBetUK"]
START_DATE = "2025-11-24"
WELCOME_OFFER_TYPE = "Sports"
BONUS_PATTERN = "%VB_FB_PRO_FBD_MAS_AUT_1%"

In [ ]:
#@title SQL Query
query = """
WITH FBB_Users AS (
  SELECT
    A.userID,
    A.Domain,
    DATE(acquisitionDateTime) AS RegDate,
    DATE(A.reward.createdDateTime) AS BonusGivenDate,
    creationDate,
    DATE_TRUNC(creationDate, ISOWEEK) AS week,
    turnover.local AS stake,
    betOddsPlaced AS odds,
    ggr.local AS ggr,
    theoGGR.local AS theoggr,
    betStatus,
    combinations
  FROM `ls-africa-data-eu-live.lithium_warehouse.PromotionPlayerReward` A
  LEFT JOIN `ls-africa-data-eu-live.lithium_warehouse.BetSnapshot` B
    ON A.userID = B.UserID
   AND DATE(A.reward.redeemedDateTime) = B.CreationDate
   AND bonus.name = reward.name
  LEFT JOIN `ls-africa-data-eu-live.lithium_warehouse.User` C
    ON A.UserID = C.UserID
  WHERE NOT isTest
    AND bonus.name LIKE '%VB_FB_PRO_FBD_MAS_AUT_1%'
    AND isFreeBet
),

WO_BonusCodes AS (
  SELECT
    string_field_1 AS domain,
    string_field_2 AS product,
    string_field_8 AS WelcomeOffer,
    string_field_14 AS BonusCode,
    CASE
      WHEN string_field_3 LIKE '%Convergence%' THEN 'Convergence'
      WHEN REGEXP_CONTAINS(LOWER(string_field_2), r'(slots|casino)') THEN 'Casino'
      WHEN string_field_2 LIKE '%Sports%' THEN 'Sports'
      WHEN string_field_2 LIKE '%Mixed%' THEN 'Hybrid'
    END AS WelcomeOfferType
  FROM `gamesys-eu-live-sbtech-insight.SourabhNagesh.Acquisition Sign Up Codes`
),

RegWeekFBB AS (
  SELECT
    DATE_TRUNC(DATE(acquisitionDateTime), ISOWEEK) AS RegWeek,
    DATE(P2PDateTime) AS P2PDate,
    u.userID,
    u.domain,
    COUNT(CASE WHEN betStatus NOT IN ('Cancelled') THEN u.userID END) AS Used_FBB,
    COUNT(DISTINCT CASE
      WHEN LOWER(accountStatusReason) IN ('duplicated account','fraud')
        OR (SelfieIDRequested IS NOT NULL AND SelfieIDCompleted IS NULL)
      THEN u.userID END
    ) AS Bonus_Abuse
  FROM `ls-africa-data-eu-live.lithium_warehouse.User` u
  LEFT JOIN FBB_Users fbb
    ON fbb.userID = u.userID
   AND DATE_TRUNC(fbb.BonusGivenDate, ISOWEEK) = DATE_TRUNC(DATE(acquisitionDateTime), ISOWEEK)
  LEFT JOIN WO_BonusCodes bc
    ON bc.BonusCode = u.signupparameters.promoCode
  LEFT JOIN `gamesys-eu-live-sbtech-insight.Data.SelfieID` s
    ON s.userID = u.userID
   AND s.domain = u.domain
  WHERE NOT isTest
    AND u.domain IN UNNEST(@domain)
    AND DATE(acquisitionDateTime) >= @start_date
    AND WelcomeOfferType = @welcome_offer_type
  GROUP BY ALL
)

SELECT
  RegWeek,
  domain,
  Used_FBB,
  Bonus_Abuse,
  COUNT(DISTINCT userID) AS Regs,
  COUNT(DISTINCT CASE WHEN P2PDate IS NOT NULL THEN userID END) AS P2Ps
FROM RegWeekFBB
GROUP BY ALL
ORDER BY 1,2,3
"""


In [ ]:
job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ArrayQueryParameter("domain", "STRING", DOMAIN),
        bigquery.ScalarQueryParameter("start_date", "DATE", START_DATE),
        bigquery.ScalarQueryParameter("welcome_offer_type", "STRING", WELCOME_OFFER_TYPE),
        bigquery.ScalarQueryParameter("bonus_pattern", "STRING", BONUS_PATTERN),
    ]
)

df = client.query(query, job_config=job_config).to_dataframe()
df


,RegWeek,domain,Used_FBB,Bonus_Abuse,Regs,P2Ps
0,2025-11-24,LiveScoreBetUK,0,1,379,135
1,2025-11-24,LiveScoreBetUK,0,0,1666,962
2,2025-11-24,VirginBet,0,1,873,43
3,2025-11-24,VirginBet,0,0,1296,730
4,2025-11-24,VirginBet,1,1,630,11
5,2025-11-24,VirginBet,1,0,376,311
6,2025-12-01,LiveScoreBetUK,0,1,1092,282
7,2025-12-01,LiveScoreBetUK,0,0,2380,1380
8,2025-12-01,VirginBet,0,1,2321,127
9,2025-12-01,VirginBet,0,0,2401,1218


In [ ]:
#@title Excel Output
OUTPUT_FILE = "fbb_analysis.xlsx"

with pd.ExcelWriter(OUTPUT_FILE, engine="openpyxl") as writer:
    df.to_excel(writer, sheet_name="Raw", index=False)
    df.to_excel(writer, sheet_name="Raw2", index=False)

OUTPUT_FILE

#To show where the file is saved.
import os
os.path.abspath(OUTPUT_FILE)


'/content/fbb_analysis.xlsx'

In [ ]:
#This is test, would the changes be reflected?